<a href="https://colab.research.google.com/github/elifyadairem/Vehicle-Licence-System/blob/main/FinalSon1ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# YOLO
!pip install ultralytics

# OCR
!pip install easyocr

# VLM ve Dataset indirme
!pip install transformers torch torchvision "fiftyone[desktop]"

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
import os


for ds_name in fo.list_datasets():
    if "coco" in ds_name or "traffic" in ds_name:
        print(f"Eski dataset siliniyor: {ds_name}")
        fo.delete_dataset(ds_name)


print("Veri seti sıfırdan indiriliyor, lütfen bekleyin...")

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    label_types=["detections"],
    classes=["car", "bus", "truck"],
    max_samples=500,
    shuffle=True,
    dataset_name="traffic_project_v1",
    drop_existing_dataset=True
)


image_paths = dataset.values("filepath")

print("-" * 30)
if len(image_paths) > 0:
    print(f"✅ BAŞARILI! Toplam {len(image_paths)} görsel indirildi.")
    print(f"İlk görsel yolu: {image_paths[0]}")
else:
    print("⚠️ HATA: Hala görsel indirilemedi.")

Veri seti sıfırdan indiriliyor, lütfen bekleyin...


INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Sufficient images already downloaded


...

 100% |█████████████████| 500/500 [11.0s elapsed, 0s remaining, 63.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 500/500 [11.0s elapsed, 0s remaining, 63.4 samples/s]      


Dataset 'traffic_project_v1' created


INFO:fiftyone.zoo.datasets:Dataset 'traffic_project_v1' created


------------------------------
✅ BAŞARILI! Toplam 500 görsel indirildi.
İlk görsel yolu: /root/fiftyone/coco-2017/validation/data/000000367680.jpg


In [3]:
import cv2
import torch
from ultralytics import YOLO
import easyocr
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


print("Modeller yükleniyor...")


yolo_model = YOLO('yolov8n.pt')

# COCO Sınıf ID'leri: 2: Car, 5: Bus, 7: Truck
TARGET_CLASSES = [2, 5, 7]
CLASS_NAMES = {2: 'Car', 5: 'Bus', 7: 'Truck'}

# OCR
reader = easyocr.Reader(['en'], gpu=True)

# C) VLM (BLIP)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
vlm_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda")

print("Tüm modeller hazır!")


# İzin verilen plaka
ALLOWED_PLATES = [ "16ACD433"]



def get_vlm_description(image_cv2):
    """Görseli VLM'e verip betimleme alır."""

    img_rgb = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2RGB)
    raw_image = Image.fromarray(img_rgb)

    inputs = processor(raw_image, return_tensors="pt").to("cuda")
    out = vlm_model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

def read_license_plate(image_crop):
    """Kırpılmış araç görselinden yazı okur."""


    result = reader.readtext(image_crop, detail=0)

    text = "".join(result).replace(" ", "").upper()
    return text

def check_access(vehicle_type, plate_text):
    """KAPI KONTROL MANTIĞI"""
    status = "RED"
    message = ""


    if vehicle_type == "Truck":
        status = "DENIED"
        message = "Kamyon girişi yasak!"
        return status, message



    is_allowed = False
    for allowed in ALLOWED_PLATES:

        if len(plate_text) > 3 and (plate_text in allowed or allowed in plate_text):
            is_allowed = True
            break

    if is_allowed:
        status = "ALLOWED"
        message = f"Plaka ({plate_text}) onaylı. Kapı açılıyor."
    else:
        status = "DENIED"
        message = f"Plaka ({plate_text}) sistemde kayıtlı değil!"

    return status, message



def run_gate_system(img_path):
    frame = cv2.imread(img_path)
    if frame is None: return


    results = yolo_model(frame, verbose=False)

    detections = results[0].boxes

    detected_vehicle = False

    for box in detections:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])

        if cls_id in TARGET_CLASSES and conf > 0.5:
            detected_vehicle = True
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            vehicle_type = CLASS_NAMES[cls_id]


            vehicle_crop = frame[y1:y2, x1:x2]


            description = get_vlm_description(vehicle_crop)


            plate_text = read_license_plate(vehicle_crop)
            if len(plate_text) < 3:
                plate_text = "UNKNOWN" # Okuyamazsa


            access_status, access_msg = check_access(vehicle_type, plate_text)


            color = (0, 255, 0) if access_status == "ALLOWED" else (0, 0, 255)


            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)


            label = f"{vehicle_type} | {access_status}"
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)


            print(f"\n--- TESPİT EDİLEN ARAÇ: {vehicle_type} ---")
            print(f"VLM Betimlemesi: {description}")
            print(f"Okunan Plaka (OCR): {plate_text}")
            print(f"KARAR: {access_msg}")

    if detected_vehicle:

        plt.figure(figsize=(10,6))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()
    else:
        print("Görselde Otobüs, Kamyon veya Araba bulunamadı.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Modeller yükleniyor...


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Tüm modeller hazır!


In [1]:
from google.colab import files
import os

print("Lütfen test etmek istediğiniz görselleri yükleyin...")


uploaded = files.upload()

print(f"\nToplam {len(uploaded)} görsel yüklendi. Analiz başlıyor...\n")
print("="*50)


for filename in uploaded.keys():

    img_path = os.path.abspath(filename)

    print(f"\n>>> Dosya İşleniyor: {filename}")


    try:
        run_gate_system(img_path)
    except Exception as e:
        print(f"HATA: {filename} işlenirken bir sorun oluştu.")
        print(f"Hata Detayı: {e}")

    print("-" * 50)

Lütfen test etmek istediğiniz görselleri yükleyin...


KeyboardInterrupt: 